In [1]:
import pandas as pd
import os
import json

In [2]:
DISCO_REPORT_FOLDER = 'Disco Report'

DATASET=['helpdesk', 'Sepsis', 'HospitalBilling', 'bpic11', 'bpic12', 'bpic13c', 'bpic13i', 'bpic13o', 'bpic15_1', 'bpic15_2', 'bpic15_3', 'bpic15_4', 'bpic15_5', 'bpic17a', 'bpic17o', 'bpic19']

REPORT_TYPE={'Activity', 'Cases', 'Variants'}

dirname = os.getcwd()
report_dir_path = os.path.join(dirname, DISCO_REPORT_FOLDER)

In [3]:
def readReport(log, type):
# Type: Cases, Activity, Variants

    filename = type + " in " + log +".csv"
    report_name = os.path.join(report_dir_path, filename)
    report_csv = pd.read_csv(report_name ,sep=',', low_memory=False)

    return report_csv

In [4]:
NUM_CASES={}
NUM_ACTIVITY={}
UNIQUE_ACTIVITY={}
NUM_VARIANTS={}
CASES_LIST = []
ACTIVITY_LIST = []
VARIANTS_LIST = []

In [5]:
for log in DATASET:
    for report in REPORT_TYPE:
        # Read all report from Disco
        locals()[log + '_' + report] = readReport(log, report)

        if report == 'Cases':
            # total number of cases
            locals()[log + '_num_cases'] = len(locals()[log + '_' + report])
            NUM_CASES[log] = str(locals()[log + '_num_cases'])
            # NUM_CASES[log] = locals()[log + '_num_cases']


            # tital number of events
            locals()[log + '_num_activity'] = (locals()[log + '_' + report])['Events'].sum()
            NUM_ACTIVITY[log] = str(locals()[log + '_num_activity'])
            # NUM_ACTIVITY[log] = locals()[log + '_num_activity']
            
            locals()[log + '_' + report + '_new'] = locals()[log + '_' + report].assign(Dataset=log)
            CASES_LIST.append(locals()[log + '_' + report + '_new'])

        if report == 'Activity':
            # number of unique activities
            locals()[log + '_unique_activity'] = len(locals()[log + '_' + report])
            UNIQUE_ACTIVITY[log] = str(locals()[log + '_unique_activity'])
            # UNIQUE_ACTIVITY[log] = locals()[log + '_unique_activity']
            
            locals()[log + '_' + report + '_new'] = locals()[log + '_' + report].assign(Dataset=log)
            ACTIVITY_LIST.append(locals()[log + '_' + report + '_new'])

        if report == 'Variants':
            # total number of variants
            locals()[log + '_num_variants'] = len(locals()[log + '_' + report])
            NUM_VARIANTS[log] = str(locals()[log + '_num_variants'])
            # NUM_VARIANTS[log] = locals()[log + '_num_variants']

            locals()[log + '_' + report + '_new'] = locals()[log + '_' + report].assign(Dataset=log)
            VARIANTS_LIST.append(locals()[log + '_' + report + '_new'])
    

In [6]:
# Export dataset for variants
pd.concat(CASES_LIST).to_json('processed_data/totalCases.json', orient="records")

pd.concat(ACTIVITY_LIST).to_json('processed_data/totalActivity.json', orient="records")

pd.concat(VARIANTS_LIST).to_json('processed_data/totalVariants.json', orient="records")

In [7]:
with open("processed_data/numCases.json", "w") as outfile: 
    json.dump(NUM_CASES, outfile)

with open("processed_data/numActivity.json", "w") as outfile: 
    json.dump(NUM_ACTIVITY, outfile)

with open("processed_data/uniqueActivity.json", "w") as outfile: 
    json.dump(UNIQUE_ACTIVITY, outfile)

with open("processed_data/numVariants.json", "w") as outfile: 
    json.dump(NUM_VARIANTS, outfile)